In [ ]:
import sys

sys.argv.append('--source_catalog_table')
# TODO: set name of the source table name in the glue catalog 
sys.argv.append('dataset-name')

sys.argv.append('--source_catalog_database')
# TODO: set name of the source database name in the glue catalog 
sys.argv.append('dataplatform-stg-refined-zone-database')

sys.argv.append('--s3_bucket_target')
# TODO: set S3 location where the transformed data will be saved
sys.argv.append('s3://bucket-name/path/to/data')

In [ ]:
import sys
from awsglue.transforms import *
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.dynamicframe import DynamicFrame
from awsglue.job import Job
from awsglue.utils import getResolvedOptions
from pyspark.sql.functions import rank, col, trim, when, max
import pyspark.sql.functions as F
# from helper.helpers import get_glue_env_var,

def get_glue_env_var(key, default="none"):
    if f'--{key}' in sys.argv:
        return getResolvedOptions(sys.argv, [key])[key]
    else:
        return default

glueContext = GlueContext(SparkContext.getOrCreate())
job = Job(glueContext)

source_catalog_table = get_glue_env_var('source_catalog_table', '')
source_catalog_database = get_glue_env_var('source_catalog_database', '')
target_bucket = get_glue_env_var('s3_bucket_target', '')

In [ ]:
# Load data from glue catalog
data_source = glueContext.create_dynamic_frame.from_catalog(
    name_space=source_catalog_database,
    table_name=source_catalog_table
)

# convert to a data frame
df = data_source.toDF()

In [ ]:
# Transform data 

In [ ]:
# Convert data frame to dynamic frame 
dyanmic_frame = DynamicFrame.fromDF(df, glueContext, "target_data_to_write")

# Write the data to S3
parquet_data = glueContext.write_dynamic_frame.from_options(
    frame=dyanmic_frame,
    connection_type="s3",
    format="parquet",
    connection_options={"path": target_bucket, "partitionKeys": ["import_year", "import_month", "import_day", "import_date"]},
    transformation_ctx="target_data_to_write")